In [8]:
import logging
logging.root.setLevel(logging.INFO)


In [9]:
import random

from xcs import XCSAlgorithm
from xcs.scenarios import Scenario

In [10]:
import random

from xcs.scenarios import Scenario
from xcs.bitstrings import BitString

class HaystackProblem(Scenario):
    
    def __init__(self, training_cycles=1000, input_size=50):
        self.input_size = input_size
        self.possible_actions = (True, False)
        self.initial_training_cycles = training_cycles
        self.remaining_cycles = training_cycles
        self.needle_index = random.randrange(input_size)
        self.needle_value = None

    def reset(self):
        self.remaining_cycles = self.initial_training_cycles
        haystack = BitString.random(self.input_size)
        self.needle_value = haystack[self.needle_index]
        
        sense_string = str(self.sense())
        raw_state = [str(s) for s in sense_string]
        return raw_state
        
    # XCS Hosford42 functions
    @property
    def is_dynamic(self):
        return False
        
    def get_possible_actions(self):
        return self.possible_actions
        
    def more(self):
        return self.remaining_cycles > 0
    
    def sense(self):
        haystack = BitString.random(self.input_size)
        self.needle_value = haystack[self.needle_index]
        return haystack
    
    def execute(self, action):
        self.remaining_cycles -= 1
        return action == self.needle_value

    # XCS Pyalcs functions
    def step(self, action):
        done = not self.execute(action)
        
        haystack = self.sense()
        sense_string = str(haystack)
        raw_state = [str(s) for s in sense_string]
        
        self.needle_value = haystack[self.needle_index]
        reward = action == self.needle_value
        return raw_state, reward, done, _

In [11]:
training_cycles = 5000
input_size = 50
logging.root.setLevel(logging.INFO)
scenario = HaystackProblem(training_cycles, input_size)

In [12]:
algorithm = XCSAlgorithm()
algorithm.exploration_probability = .1
algorithm.discount_factor = 0
algorithm.do_ga_subsumption = True
algorithm.do_action_set_subsumption = True

In [13]:
model = algorithm.new_model(scenario)
model.run(scenario, learn=True)


In [14]:
print(model)


01#1#1###0##011010001###00##0#0011##0011#0#1##0110 => False
    Time Stamp: 4608
    Average Reward: 0.0
    Error: 0.0
    Fitness: 8.5e-07
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
1##1#1011#10010#0#011#0#1##0###01#0#100###1##1#001 => False
    Time Stamp: 3721
    Average Reward: 1.0
    Error: 0.0
    Fitness: 8.5e-07
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
11##0#0#1101011111110000#0#0#1000110100##1#0###11# => True
    Time Stamp: 4344
    Average Reward: 1.0
    Error: 0.0
    Fitness: 8.5e-07
    Experience: 0
    Action Set Size: 1
    Numerosity: 1
1110#0#00010##011#1##001011#011#10#01#00##000####1 => False
    Time Stamp: 4215
    Average Reward: 0.0
    Error: 0.0
    Fitness: 0.15000850000000002
    Experience: 1
    Action Set Size: 1.0
    Numerosity: 1
00011#1#1#01###100#0#1101#001#1101100#1#0111101#01 => True
    Time Stamp: 4272
    Average Reward: 0.0
    Error: 0.0
    Fitness: 0.15000850000000002
    Experience: 1
    Action Set Si

In [15]:
print(len(model))

200


In [28]:
for rule in model:
    if rule.fitness > .05 and rule.experience >= 1:
        print(rule.condition, '=>', rule.action, ' [%.5f]' % rule.fitness)

#11##1010110##0##01111000011#1001#10##10#0110001#1 => True  [0.15001]
01111100#011#0000101000101#01#0#0110##100#01#0#000 => False  [0.15001]
101#0##0##1111#0#01#1111##100110011101#0#1#1#01001 => False  [0.15001]
###00#0#011###100#010##011110000#0#0##100#1#011111 => True  [0.15001]
1110#0#00010##011#1##001011#011#10#01#00##000####1 => False  [0.15001]
##1000011#100111####11###00#010100#0##0000#01####0 => False  [0.15001]
00011#1#1#01###100#0#1101#001#1101100#1#0111101#01 => True  [0.15001]
##00#10#0111##0011#1###0010#1#0##11010#11110100000 => True  [0.15001]
0010100##0011#111#101##00000000#0#0101#0###1010#01 => True  [0.15001]
01110###10110111100000##00001001000101010011#0001# => True  [0.15001]
1011001#000##01001#1#1#1000#11##10#000#110#0#0##01 => False  [0.15001]
110#1##1100001####0#1010#1#10#11#11#0010001##111#0 => True  [0.15001]
0010#0##1#1##0000#1111#1#10#1#1110#01##101#10#011# => False  [0.15001]
0#01#1####1000###100##10#000010###101#1010#0#00##0 => False  [0.15001]
011##11010#00

In [17]:
from lcs.agents.xcs import XCS, Configuration

cfg = Configuration(number_of_actions=4,
                    gamma=0,
                    metrics_trial_frequency=5,
                    covering_wildcard_chance=0.9
                    )


In [18]:
agent = XCS(cfg)
explore_population, explore_metrics = agent.explore(scenario, training_cycles, False)

In [19]:
for rule in explore_population:
    print(rule)


Cond:#1#1111000#100111011111110110111100010#10101010001 - Act:0 - Num:1 [fit: 0.000, exp: 0.00, pred: 0.000]
Cond:01010001111010###01#00#10#0100000000011000#0011110 - Act:1 - Num:1 [fit: 0.050, exp: 1.00, pred: 0.529]
Cond:1111010110111#11110#001000###1#00#10##1#0001011011 - Act:1 - Num:1 [fit: 0.000, exp: 0.00, pred: 0.000]
Cond:111#0100101111001#00111101100010#011#1001010111000 - Act:1 - Num:1 [fit: 0.000, exp: 0.00, pred: 0.000]
Cond:0#11#11000##0101#00100111001#110000101010010111000 - Act:0 - Num:1 [fit: 0.050, exp: 1.00, pred: 0.440]
Cond:01000001011000101111011100011011010111010010001010 - Act:1 - Num:1 [fit: 0.000, exp: 0.00, pred: 0.000]
Cond:000000010011110#11010000#0111110010#11##1110#11011 - Act:2 - Num:1 [fit: 0.000, exp: 0.00, pred: 0.000]
Cond:0000100011001000000000110010#010100000111100111111 - Act:2 - Num:1 [fit: 0.000, exp: 0.00, pred: 0.000]
Cond:1#11001011011100#01#0100#1#010100011010#01#11#01#1 - Act:3 - Num:1 [fit: 0.050, exp: 1.00, pred: 0.000]
Cond:11#0010011#011

In [20]:
print(len(explore_population))

200


In [31]:
for rule in explore_population:
    if rule.fitness >= .05 and rule.experience >= 1:
        print(rule.condition, '=>', rule.action, ' [%.5f]' % rule.fitness)

01010001111010###01#00#10#0100000000011000#0011110 => 1  [0.05000]
0#11#11000##0101#00100111001#110000101010010111000 => 0  [0.05000]
1#11001011011100#01#0100#1#010100011010#01#11#01#1 => 3  [0.05000]
0001#101011001001101010001##011101110111#110000111 => 3  [0.05000]
111#011110001111010010011#101011101#01100011111010 => 2  [0.05000]
00#00#1000110111#110001011010#0010101100110001011# => 3  [0.05000]
01010111001#10##00101000#0101111111111111011100#01 => 3  [0.05000]
0#010#1#0##110#01011#011#10000101010011111##101110 => 2  [0.05000]
0101010#110001000000111000000101#00000001010000101 => 0  [0.05000]
01110101101011001101#00010111100001010110001010110 => 3  [0.05000]
10#1001010001000001010110#100#0101001#11011000100# => 0  [0.05000]
010001#0000#10110#101#11000#11000##1#0000011100111 => 3  [0.05000]
10110#000010001110011#01000101#11001111101000#11#1 => 3  [0.05000]
111#0111100000011011#00#11001011000001001000111101 => 2  [0.05000]
00100#10111000011001011111011#0001111001011#001101 => 3  [0.05

#### Conclusions
Both XCS implementations provide similar looking population. Both has population size of 200, and both provide similarly looking classifiers.
Main difference being differenty amount of classifiers with fitness above 0.05, which can easily be explained by algorithms using different RP.